In [1]:
!pip install dfply
!pip install plotly
!pip install xhtml2pdf
import pandas as pd
import numpy as np
import math
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statistics 
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import pearsonr
import plotly.graph_objects as go
import warnings
from scipy.stats import shapiro
from datetime import date, timedelta
from IPython.display import Image

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
transfers = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Transfers_Clean.pkl")

In [0]:
markval = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/markval_Clean.pkl")

In [0]:
continents = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/continents.pkl")

In [0]:
leagues_teams = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/teams_leagues_Clean.pkl")

In [0]:
players = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")

In [7]:
transfers.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 79891 entries, 2011-07-01 to 2018-01-04
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   tm_id   79891 non-null  int64   
 1   from    79891 non-null  object  
 2   to      79891 non-null  object  
 3   fee     79891 non-null  float64 
 4   mv      69690 non-null  float64 
 5   season  79891 non-null  category
 6   loan    79891 non-null  bool    
dtypes: bool(1), category(1), float64(2), int64(1), object(2)
memory usage: 3.8+ MB


In [0]:
leagues_teams = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/teams_leagues_Clean.pkl")
leagues_teams = (leagues_teams >> group_by(X["club"],X["league"],X["country"]) >> 
 summarize(club = X.club.unique()[0], league = X.league.unique()[0])
 )

In [30]:
leagues_teams.league.unique()

array(['Bundesliga', '2. Bundesliga', '3. Liga', 'Ligue 2', 'Série A',
       'Superligaen', 'Serie A', 'Serie B', 'LaLiga2', 'Eredivisie',
       '2. Liga Inter - Gr. 3', 'Salzburger Liga', 'Premier League',
       'League One', 'Allsvenskan', 'Ligue 1', 'Eliteserien',
       'Premiership', 'Professional League', 'A-League', 'Süper Lig',
       'Liga DIMAYOR I', 'Ekstraklasa', 'Superliga', 'LaLiga', 'MLS',
       'Liga MX Clausura', 'Super League', 'Championship', 'Liga NOS',
       'League Two', 'K League 1', 'Jupiler Pro League', 'J1 League'],
      dtype=object)

In [0]:
transfers = transfers.reset_index()
transfers = transfers.rename(str.lower, axis='columns')

In [10]:
transfers

,date,tm_id,from,to,fee,mv,season,loan
0,2011-07-01,17259,FC Schalke 04,Bayern Munich,30000000.0,28000000.0,11/12,False
1,2015-07-01,40680,VfB Stuttgart,Bayern Munich,3500000.0,3500000.0,15/16,False
2,1998-07-01,40680,Schornbach Yth.,Stuttgart Yth.,0.0,NaN,98/99,False
3,1994-07-01,40680,Lichtenw. Yth.,Schornbach Yth.,0.0,NaN,94/95,False
4,2014-07-01,336307,Deggendorf Yth.,FC Bayern Münche,0.0,NaN,14/15,False
...,...,...,...,...,...,...,...,...
79886,2016-07-24,365861,Maccabi Haifa,Slovan Liberec,0.0,0.0,16/17,True
79887,2017-06-30,365861,Slovan Liberec,Maccabi Haifa,0.0,250000.0,16/17,True
79888,2019-01-01,198395,FC Zlin,Viktoria Plzen,0.0,750000.0,18/19,False
79889,2012-07-01,198395,Toulouse B,Esperance,0.0,NaN,12/13,False


In [0]:
def get_league(club):
  global leagues_teams
  all_clubs = leagues_teams.club.unique()
  league = math.nan
  try:
    if club in all_clubs:
      league = leagues_teams[leagues_teams.club == club].league.values[0]
  except Exception as e:
    print(str(e))
    pass
  return league

In [0]:
def detailed_transfers(transfers, markval):
  markval["year"] = markval.index
  markval.year = markval.year.apply(lambda x : x.year)
  markval = markval.groupby('tm_id').apply(lambda x: x.sort_values('year', ascending = False))
  markval = markval.droplevel('tm_id')
  markval["last_year_club"] = markval.groupby('tm_id').pipe(lambda x: x.club.shift(-1))
  markval["last_year_league"] = markval.groupby('tm_id').pipe(lambda x: x.league.shift(-1))
  markval = markval[markval.last_year_club.notnull()]
  transfer_vals = markval >> mask(X.club != X.last_year_club)
  transfer_vals = transfer_vals.groupby(["tm_id","season"]).apply(lambda x: x.drop_duplicates(["year"],"last")) 
  transfer_vals = transfer_vals.droplevel('tm_id')
  transfer_vals = transfer_vals.droplevel("season")
  transfer_vals = transfer_vals.reset_index()
  return pd.merge(transfers,
                  transfer_vals[['tm_id','last_year_league', 'league','season','last_year_club']],
                  on=['tm_id','season'])

In [0]:
transfers_detailed = detailed_transfers(transfers, markval)

In [0]:
transfers_detailed = transfers_detailed.drop_duplicates() 

In [0]:
transfers_detailed.rename({"last_year_league":"from_league",
                                             "league":"to_league"}, 
                          axis ="columns", inplace = True) 

In [0]:
def get_country(league):
  global leagues_teams
  all_leagues = leagues_teams.league.unique()
  country = math.nan
  try:
    if league in all_leagues:
      country = leagues_teams[leagues_teams.league == league].country.values[0]
  except Exception as e:
    print(str(e))
    pass
  return country

In [0]:
transfers_detailed["from_country"] = transfers_detailed["from_league"].apply(get_country)

In [0]:
transfers_detailed["to_country"] = transfers_detailed["to_league"].apply(get_country)

In [0]:
def get_continent(country):
  global continents
  all_countries = continents.country.unique()
  continent = math.nan
  try:
    if country in all_countries:
      continent = continents[continents.country == country].continent.values[0]
  except Exception as e:
    print(str(e))
    pass
  return continent

In [0]:
transfers_detailed["from_cont"] = transfers_detailed["from_country"].apply(get_continent)

In [0]:
transfers_detailed["to_cont"] = transfers_detailed["to_country"].apply(get_continent)

In [0]:
def get_age(age_delta):
  result = math.nan
  try:
    result = int(age_delta / timedelta(days=365.2425))
  except:
    pass
  return result   

In [0]:
players_tm = players.reset_index()
transfers_detailed = pd.merge(transfers_detailed,
                players_tm[['tm_id','name', 'dob', 'nationality', 'continent','field_position','main_field_position']],
                on='tm_id')
transfers_detailed["age"] = (transfers_detailed.date - transfers_detailed.dob)
transfers_detailed.age = transfers_detailed.age.apply(get_age) 

In [0]:
def summer_winter(date):
  month = date.month
  if 0 <= month <= 2:
    return "winter"
  elif 6 <= month <= 8:
    return "summer"
  else:
    return "mid_seson"

In [0]:
transfers_detailed["type"] = transfers_detailed.date.apply(summer_winter)
transfers_detailed["type"] = transfers_detailed["type"].astype('category') 

In [0]:
transfers_detailed.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/Transfers_Network.pkl")

In [0]:
transfers_detailed = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Transfers_Network.pkl")

In [39]:
transfers_detailed

,date,tm_id,from,to,fee,mv,season,loan,from_league,to_league,last_year_club,from_country,to_country,from_cont,to_cont,name,dob,nationality,continent,field_position,main_field_position,age,type
0,2011-07-01,17259,FC Schalke 04,Bayern Munich,30000000.0,28000000.0,11/12,False,Bundesliga,Bundesliga,FC Schalke 04,Germany,Germany,EU,EU,Manuel Neuer,1986-03-27,Germany,EU,GK,goalkeepers,25.0,summer
1,2015-07-01,40680,VfB Stuttgart,Bayern Munich,3500000.0,3500000.0,15/16,False,2. Bundesliga,Bundesliga,VfB Stuttgart,Germany,Germany,EU,EU,Sven Ulreich,1988-08-03,Germany,EU,GK,goalkeepers,26.0,summer
2,2019-07-01,281963,Atlético Madrid,Bayern Munich,80000000.0,70000000.0,19/20,False,LaLiga,Bundesliga,Atlético Madrid,Spain,Germany,EU,EU,Lucas Hernández,1996-02-14,France,EU,CB,defenders,23.0,summer
3,2017-07-01,166601,TSG Hoffenheim,Bayern Munich,20000000.0,25000000.0,17/18,False,Bundesliga,Bundesliga,TSG 1899 Hoffenheim,Germany,Germany,EU,EU,Niklas Süle,1995-09-03,Germany,EU,CB,defenders,21.0,summer
4,2007-08-21,26485,Hertha BSC,Hamburger SV,1100000.0,750000.0,07/08,False,Bundesliga,2. Bundesliga,Hertha BSC,Germany,Germany,EU,EU,Jérôme Boateng,1988-09-03,Germany,EU,CB,defenders,18.0,summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33206,2011-12-01,62800,Slavia Prague,FK Jablonec,0.0,1500000.0,11/12,True,3. Liga,2. Bundesliga,1.FC Kaiserslautern,Germany,Germany,EU,EU,Adam Hlousek,1988-12-20,Czech Republic,EU,LB,defenders,22.0,mid_seson
33207,2011-08-01,62800,FK Jablonec,Slavia Prague,0.0,1500000.0,11/12,True,3. Liga,2. Bundesliga,1.FC Kaiserslautern,Germany,Germany,EU,EU,Adam Hlousek,1988-12-20,Czech Republic,EU,LB,defenders,22.0,summer
33208,2012-01-01,62800,FK Jablonec,1.FC Nuremberg,1000000.0,1200000.0,11/12,False,3. Liga,2. Bundesliga,1.FC Kaiserslautern,Germany,Germany,EU,EU,Adam Hlousek,1988-12-20,Czech Republic,EU,LB,defenders,23.0,winter
33209,2014-07-01,62800,1.FC Nuremberg,VfB Stuttgart,1450000.0,1500000.0,14/15,False,2. Bundesliga,2. Bundesliga,1.FC Nuremberg,Germany,Germany,EU,EU,Adam Hlousek,1988-12-20,Czech Republic,EU,LB,defenders,25.0,summer
